In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
sample_submission = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')

In [3]:
sample_submission

,time_id,row_id,target
0,26290,478_0_0,1
1,26290,478_0_1,1
2,26290,478_0_2,1
3,26290,478_0_3,1
4,26290,478_0_4,1
...,...,...,...
32995,26454,480_540_195,1
32996,26454,480_540_196,1
32997,26454,480_540_197,1
32998,26454,480_540_198,1


# Load Data

In [4]:
train_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
test_df  = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')

In [5]:
print('Shape of training data =', train_df.shape)
print('Shape of testing data  =', test_df.shape)

Shape of training data = (5237980, 17)
Shape of testing data  = (33000, 16)


In [6]:
train_df.describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,5.237980e+06,5.237980e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237760e+06,2.343638e+06,2.380800e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237892e+06,5.237980e+06
mean,9.928856e+01,2.415100e+02,2.700000e+02,5.715293e+06,-1.189619e-02,9.999955e-01,4.510025e+07,1.001713e+00,9.996601e-01,9.997263e-01,5.181359e+04,1.000264e+00,5.357568e+04,9.999920e-01,-4.756125e-02,1.331005e+04
std,5.787176e+01,1.385319e+02,1.587451e+02,2.051591e+07,8.853374e-01,2.532497e-03,1.398413e+08,7.214705e-01,1.216920e-02,2.499345e-03,1.114214e+05,2.510042e-03,1.293554e+05,2.497509e-03,9.452860e+00,7.619271e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,9.352850e-01,4.316610e+03,7.700000e-05,7.869880e-01,9.349150e-01,0.000000e+00,9.398270e-01,0.000000e+00,9.380080e-01,-3.852898e+02,0.000000e+00
25%,4.900000e+01,1.220000e+02,1.300000e+02,8.453415e+04,-1.000000e+00,9.987630e-01,5.279575e+06,9.963320e-01,9.971000e-01,9.985290e-01,7.374720e+03,9.990290e-01,7.823700e+03,9.987810e-01,-4.559755e+00,6.729000e+03
50%,9.900000e+01,2.420000e+02,2.700000e+02,1.113604e+06,0.000000e+00,9.999670e-01,1.288264e+07,9.998830e-01,9.998890e-01,9.997280e-01,2.196900e+04,1.000207e+00,2.301792e+04,9.999970e-01,-6.020069e-02,1.334500e+04
75%,1.490000e+02,3.610000e+02,4.100000e+02,4.190951e+06,1.000000e+00,1.001174e+00,3.270013e+07,1.003318e+00,1.002590e+00,1.000905e+00,5.583168e+04,1.001414e+00,5.787841e+04,1.001149e+00,4.409552e+00,1.990700e+04
max,1.990000e+02,4.800000e+02,5.400000e+02,2.982028e+09,1.000000e+00,1.077488e+00,7.713682e+09,4.379531e+02,1.309732e+00,1.077488e+00,3.028784e+07,1.077836e+00,5.440500e+07,1.077675e+00,4.460704e+02,2.645400e+04


In [7]:
train_df.head(10)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4
5,5,0,0,0.00,0,1.000635,13552875.92,NaN,NaN,0.999779,1962.72,1.000635,5647.65,1.0,6.779432,0,0_0_5
6,6,0,0,969969.40,1,1.000115,3647503.98,NaN,NaN,0.999506,6663.16,1.000283,3810.48,1.0,-2.499819,0,0_0_6
7,7,0,0,9412959.10,1,0.999818,21261245.87,NaN,NaN,0.999741,5139.20,1.000130,2570.60,1.0,-1.959801,0,0_0_7
8,8,0,0,2394875.85,1,0.999916,9473209.08,NaN,NaN,0.999022,52011.60,1.000041,2169.36,1.0,-5.970001,0,0_0_8
9,9,0,0,3039700.65,-1,1.000969,6248958.45,NaN,NaN,0.999354,6191.00,1.000646,6199.00,1.0,7.970333,0,0_0_9


In [8]:
X = train_df.loc[:, train_df.columns != 'target']
y = train_df.loc[:, train_df.columns == 'target']

### Training/Validation/Testing splt

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

### Model Training

In [11]:
X_train.select_dtypes(include='number')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id
1051948,39,98,320,6813610.49,-1,0.999693,32283026.85,0.962047,0.982373,0.999693,50985.09,1.000083,133833.58,0.999801,5422
4491080,100,413,50,0.00,0,0.998899,1029918.09,NaN,NaN,0.997623,87083.52,1.000174,93597.84,0.998854,22720
4005953,173,368,540,248934.59,1,1.000594,5397201.48,1.001235,1.001128,1.000433,73776.50,1.000594,33335.84,1.000544,20294
3533862,82,326,40,0.00,0,0.996989,159127.49,NaN,NaN,0.996989,128976.57,1.002209,176774.40,0.999198,17934
2814503,171,260,270,4802439.09,1,0.998607,8475071.64,NaN,NaN,0.998456,528.80,0.998607,39798.22,0.998458,14327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628792,42,59,180,0.00,0,0.999029,2042364.59,NaN,NaN,0.999029,25841.54,0.999865,75938.64,0.999242,3263
3793916,136,349,390,445211.87,-1,0.998549,3851367.94,0.986862,0.995578,0.998549,8166.42,0.998945,26576.61,0.998642,19234
2134542,193,198,200,6005514.66,-1,1.001923,44553669.03,NaN,NaN,1.001923,54750.08,1.002181,7779.00,1.002149,10910
3796366,186,349,510,0.00,0,0.997908,33242460.28,0.997908,0.997908,0.997729,31108.35,0.997908,25158.32,0.997828,19246


In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
X_train.select_dtypes(include='number').fillna(0)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id
1051948,39,98,320,6813610.49,-1,0.999693,32283026.85,0.962047,0.982373,0.999693,50985.09,1.000083,133833.58,0.999801,5422
4491080,100,413,50,0.00,0,0.998899,1029918.09,0.000000,0.000000,0.997623,87083.52,1.000174,93597.84,0.998854,22720
4005953,173,368,540,248934.59,1,1.000594,5397201.48,1.001235,1.001128,1.000433,73776.50,1.000594,33335.84,1.000544,20294
3533862,82,326,40,0.00,0,0.996989,159127.49,0.000000,0.000000,0.996989,128976.57,1.002209,176774.40,0.999198,17934
2814503,171,260,270,4802439.09,1,0.998607,8475071.64,0.000000,0.000000,0.998456,528.80,0.998607,39798.22,0.998458,14327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628792,42,59,180,0.00,0,0.999029,2042364.59,0.000000,0.000000,0.999029,25841.54,0.999865,75938.64,0.999242,3263
3793916,136,349,390,445211.87,-1,0.998549,3851367.94,0.986862,0.995578,0.998549,8166.42,0.998945,26576.61,0.998642,19234
2134542,193,198,200,6005514.66,-1,1.001923,44553669.03,0.000000,0.000000,1.001923,54750.08,1.002181,7779.00,1.002149,10910
3796366,186,349,510,0.00,0,0.997908,33242460.28,0.997908,0.997908,0.997729,31108.35,0.997908,25158.32,0.997828,19246


In [14]:
non_nan_idxs = y_train[~y_train.isnull().any(axis=1)].index

In [15]:
X_train = X_train.loc[non_nan_idxs]
y_train = y_train.loc[non_nan_idxs]

In [16]:
cols = list(X_train.select_dtypes(include='number').columns)
cols.remove('near_price')
cols.remove('far_price')
cols.remove('time_id')

In [17]:
lr = LinearRegression()
lr.fit(
    X_train[cols].fillna(0),
    y_train)

LinearRegression()

In [18]:
lr

LinearRegression()

# Make submission

In [19]:
preds = pd.DataFrame(lr.predict(test_df[cols]))
preds

,0
0,-0.351594
1,1.610120
2,2.094969
3,-0.540982
4,-0.592713
...,...
32995,-0.058858
32996,-0.502137
32997,0.090405
32998,0.302044


In [20]:
submission = pd.concat([test_df[['time_id', 'row_id']], preds], axis=1)
submission.columns = ['time_id', 'row_id', 'target']
submission

,time_id,row_id,target
0,26290,478_0_0,-0.351594
1,26290,478_0_1,1.610120
2,26290,478_0_2,2.094969
3,26290,478_0_3,-0.540982
4,26290,478_0_4,-0.592713
...,...,...,...
32995,26454,480_540_195,-0.058858
32996,26454,480_540_196,-0.502137
32997,26454,480_540_197,0.090405
32998,26454,480_540_198,0.302044


In [21]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [22]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
#     if counter == 0:
#         print(test.head(3))
#         print(revealed_targets.head(3))
#         print(sample_prediction.head(3))
    sample_prediction['target'] = lr.predict(test[cols])
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
